### This is where the ML happens
first import all our different dataframes and concat them all, currently 2 with data from my PC and 1 from my Laptop, hoping the model will be able to tell the difference

In [ ]:
import pandas as pd
import sqlite3
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="xgboost")

dfPC = pd.read_sql("SELECT * FROM TransformedData", sqlite3.connect("../data/transformed_outputPC.db"))
dfLaptop = pd.read_sql("SELECT * FROM TransformedData", sqlite3.connect("../data/transformed_outputLaptop.db"))
dfPCnew = pd.read_sql("SELECT * FROM TransformedData", sqlite3.connect("../data/transformed_outputPCnew.db"))


df_total = pd.concat([dfPC, dfPCnew, dfLaptop])

#### Feature test
Dataframe already has a lot of features at this point, still adding a words per minute column, filtering out extremely low wpm and high typing_duration values.

In [ ]:
import numpy as np
df_total['wpm'] = (df_total['word_count'] / df_total['typing_duration'] * 60)
df_total.loc[df_total['wpm'] < 10, 'wpm'] = np.nan
df_total.loc[df_total['typing_duration'] > 100, 'typing_duration'] = np.nan

Preparing the data for the model, message_id is not needed and user_id is what we want to predict.
20% test data

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

X = df_total.drop(columns=["user_id", "message_id"])
y = le.fit_transform(df_total['user_id'])  # y will now be numeric

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

Chose to do a XGBoost for this, initializing it and training it

In [ ]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import make_scorer, f1_score

model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)

# Use stratified k-fold to maintain class balance
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define F1 scorer
f1 = make_scorer(f1_score, average='macro')

# Cross-validate and get F1 scores
f1_scores = cross_val_score(model, X, y, scoring=f1, cv=cv, error_score='raise')
model.fit(X_train, y_train)

print("F1 scores per fold:", f1_scores)
print("Mean F1 score:", f1_scores.mean())

### In case you forget 
Precision = How many predictions of Class X where actually class X
Recall = How many instances of class X where correctly predicted (false negatives included)
F1-Score = balance between the first 2, balancing false negatives and false positives
Support = How many of Class X were in the dataset
### At this stage its a bit hard to tell still because my sample size is extremely low. (29 and 13)  Class 0 has a f1-score of 0.89 and Class 1 of 0.67, Class 1 just struggles with recall (0.54). For now,  results do not seem too bad.

In [ ]:
from sklearn.metrics import classification_report
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

Just taking a look at the averages of my 2 users out of interest

In [ ]:
user_averages = df_total.groupby('user_id').mean(numeric_only=True)
user_averages

Visualizing the importance of the features. Most important feature is the avg_dwell of all buttons while the backspacing patters don't seem to matter at all -> Since its me, thats a patter that goes across plattform I assume

In [ ]:
import matplotlib.pyplot as plt

# Print raw importances
print(model.feature_importances_)

# Plot importances
xgb.plot_importance(model)
plt.show()

Checking for the fastest words per minute in the data, not relevant

In [ ]:
df_total.sort_values(by='wpm', ascending=True).head(100)  # lowest 10 values
